# Feature Selection in der Data Science Pipeline
## *am Beispiel von medizinischen Diagnosen (Brustkrebs Diagnose)*

Wir haben den öffentlich verfügbaren Datensatz Breast Cancer Wisconsin verwendet und vom UCI Machine Learning Repository heruntergeladen. 

Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

![](./data.png)

Unser Ziel ist es, herauszufinden, welche Merkmale bei der Vorhersage von bösartigem oder gutartigem Krebs am hilfreichsten sind, und zu klassifizieren, ob der Brustkrebs gutartig oder bösartig ist.

Wir haben den öffentlich verfügbaren Datensatz Breast Cancer Wisconsin verwendet und vom UCI Machine Learning Repository heruntergeladen.

Die typische Leistungsanalyse wird durchgeführt

![](conf_matrix.png)

In [ ]:
# Bibliotheken importieren
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cf_matrix import make_confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, cross_validate
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
# Download des Krebs-Datensatzes 
import seaborn as sns
from sklearn import preprocessing
(X, y) = load_breast_cancer(return_X_y=True, as_frame=True)
# Überblick über die Daten
X

### Baseline: Entscheidungsbäumen Klassifikator

In [ ]:
# Baseline in der Performance mit Entscheidungsbäumen
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cf_matrix import make_confusion_matrix
# Daten Skalierung
t = MinMaxScaler()
t.fit(X)
X = t.transform(X)
# Einfacher binärer Klassifikator
model = XGBClassifier()
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

# Feature-Selection Methoden

Es ist üblich, statistische Maße des Korrelationstyps zwischen Eingangs- und Ausgangsvariablen als Grundlage für die Feature Selection zu verwenden.

Die Wahl der statistischen Maße hängt also stark von den Datentypen der Variablen ab.

# $\chi^2$

Für den ersten Ansatz berechnen wir die Chi-Quadrat-Statistik zwischen jedem nicht-negativen Feature und der Klasse.

Dieser Wert kann verwendet werden, um die n_features-Merkmale mit den höchsten Werten für die Test-Chi-Quadrat-Statistik aus dem Eingabe-Merkmalsvektor relativ zu den Klassen auszuwählen.

Erinnern Sie sich, dass der Chi-Quadrat-Test die Abhängigkeit zwischen stochastischen Variablen misst, so dass wir damit die Features entfernen, die am wahrscheinlichsten unabhängig von der Klasse sind und daher für die Klassifizierung irrelevant sind.

In [ ]:
# Download des Krebs-Datensatzes 
import seaborn as sns
from sklearn import preprocessing
(X, y) = load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
# Chi-Quadrat Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Ermitteln der besten k = 20 Features 
chi_best = SelectKBest(chi2, k=20).fit(X, y)
mask_features = chi_best.get_support() 
new_features = [] # The list of your K best features
for bool, feature in zip(mask_features, X.columns):
    if bool:
        new_features.append(feature)
X_new = chi_best.fit_transform(X, y)
# Überblick über die filtrierte Daten
new_features

In [ ]:
# chi-quadrat Feature Selection in der Performance mit XGBoost Modell
# Einfacher binärer Klassifikator
model = XGBClassifier()
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X_new, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

# minimum-Redundancy-Maximum-Relevance (mRMR)

Das Ziel ist es, eine Feature-Submenge auszuwählen, die die statistische Eigenschaft einer Ziel-Klassifikationsvariable am besten charakterisiert, unter der Einschränkung, dass diese Features untereinander so unähnlich wie möglich sind, aber der Klassifikationsvariable so wenig wie möglich ähnlich sind. 

Es gibt verschiedene Formen von mRMR, wobei "Relevanz" und "Redundanz" durch Mutual Information, Korrelation, t-Test/F-Test, Distanzen, etc. definiert wurden.


In [ ]:
import pymrmr

rel_feat = pymrmr.mRMR(X, 'MID', 20)
X_new = X[X.columns.intersection(rel_feat)]
rel_feat

In [ ]:
# mRMR Feature Selection in der Performance mit XGBoost Modell
# Einfacher binärer Klassifikator
model = XGBClassifier()
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X_new, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

# Vergleichende Analyse der Feature Selection Methoden

![](evaluation.png)

Über diesen Github-Link können Sie auf den gesamten Code und die Daten des Projekts zugreifen.

![](download_code.png)